In [ ]:
from neo4j import GraphDatabase

class Neo4jExporter:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def export_nodes(self):
        query = """
        WITH ['EFO', 'GO', 'LINCS', 'NCBI', 'SNOMEDCT_US', 'PUBCHEM', 'GTEXEXP', 'EXPBINS', 'IDGD', 'UBERON', 'IDGP', 'UNIPROTKB', 'HGNC'] as SAB_LIST, 
        "MATCH (t:Term)-[r1]-(cde:Code)-[r2]-(con:Concept) 
            WHERE cde.SAB in $sabList
            RETURN con.CUI as CUI, 
            CASE cde.SAB
                WHEN 'EFO' THEN 'EFO'
                when 'NCBI' then 'NCBI'
                when 'UNIPROTKB' then 'Protein'
                when 'HGNC' then 'Gene'
                when 'GO' then 'GO'
                when 'EFO' then 'EFO'
                when 'SNOMEDCT_US' then 'Disease'
                when 'PUBCHEM' then 'Compound'
                when 'EXPBINS' then 'Tissue'
                when 'UBERON' then 'Tissue'
                when 'GTEXEXP' then 'Tissue'
            end as Node_label,
            cde.SAB as SAB, cde.CODE as Code, t.name as Label, t.name as Synonyms" AS query
        CALL apoc.export.csv.query(query,"nodes.tsv", {params: {sabList: SAB_LIST}, separator: '\t'}) 
        YIELD file, nodes, relationships, properties, time
        RETURN file, nodes, relationships, properties, time
        """
        
        with self.driver.session() as session:
            result = session.run(query)
            for record in result:
                print(f"Exported to {record['file']} with {record['nodes']} nodes and {record['relationships']} relationships in {record['time']} ms")

    def export_relationships(self):
        # query = """
        # WITH ['EFO', 'GO', 'LINCS', 'NCBI', 'SNOMEDCT_US', 'PUBCHEM', 'GTEXEXP', 'EXPBINS', 'IDGD', 'UBERON', 'IDGP', 'UNIPROTKB', 'HGNC'] as SAB_LIST, 
        # "MATCH (source_code:Code)-[r1]-(source_concept:Concept)-[relationOfInterest]-(target_concept:Concept)-[r2]-(target_code:Code)
        # WHERE source_code.SAB IN $sabList AND target_code.SAB IN $sabList
        # RETURN source_concept.CUI as source, type(relationOfInterest) as relation, target_concept.CUI as target, source_code.CodeID as source_label, target_code.CodeID as target_label, relationOfInterest.SAB as SAB" as query
        # CALL apoc.export.csv.query(query, "relationships.tsv", {params: {sabList: SAB_LIST, batchSize: 100000, separator: '\t'}})
        # YIELD file, nodes, relationships, properties, time
        # RETURN file, nodes, relationships, properties, time
        # """

        query = """
        WITH ['EFO', 'GO', 'LINCS', 'NCBI', 'SNOMEDCT_US', 'PUBCHEM', 'GTEXEXP', 'EXPBINS', 'IDGD', 'UBERON', 'IDGP', 'UNIPROTKB', 'HGNC'] as SAB_LIST
        CALL apoc.export.csv.query(
        "MATCH (source_code:Code)-[r1]-(source_concept:Concept)-[relationOfInterest]-(target_concept:Concept)-[r2]-(target_code:Code)
        WHERE source_code.SAB IN $SAB_LIST AND target_code.SAB IN $SAB_LIST
        WITH source_code, target_code, source_concept, target_concept, relationOfInterest,
        CASE WHEN source_code.SAB IN ['NCBI'] THEN 'NCBI'
        WHEN source_code.SAB IN ['UNIPROTKB'] THEN 'Protein'
        WHEN source_code.SAB IN ['HGNC'] THEN 'Gene'
        WHEN source_code.SAB IN ['GO'] THEN 'GO'
        WHEN source_code.SAB IN ['EFO', 'SNOMEDCT_US'] THEN 'Disease'
        WHEN source_code.SAB IN ['PUBCHEM'] THEN 'Compound'
        WHEN source_code.SAB IN ['EXPBINS', 'UBERON', 'GTEXEXP'] THEN 'Tissue'
        ELSE 'Unknown' END as sourceLabel,
        CASE WHEN target_code.SAB IN ['NCBI'] THEN 'NCBI'
        WHEN target_code.SAB IN ['UNIPROTKB'] THEN 'Protein'
        WHEN target_code.SAB IN ['HGNC'] THEN 'Gene'
        WHEN target_code.SAB IN ['GO'] THEN 'GO'
        WHEN target_code.SAB IN ['EFO', 'SNOMEDCT_US'] THEN 'Disease'
        WHEN target_code.SAB IN ['PUBCHEM'] THEN 'Compound'
        WHEN target_code.SAB IN ['EXPBINS', 'UBERON', 'GTEXEXP'] THEN 'Tissue'
        ELSE 'Unknown' END as targetLabel
        RETURN source_concept.CUI as source, type(relationOfInterest) as relation, target_concept.CUI as target,
        sourceLabel, targetLabel, relationOfInterest.SAB as SAB",
        "relationships.tsv",
        {params: {SAB_LIST: SAB_LIST}, batchSize: 100000, separator: ','}
        ) YIELD file, nodes, relationships, properties, time
        RETURN file, nodes, relationships, properties, time;
        """
        
        with self.driver.session() as session:
            result = session.run(query)
            for record in result:
                print(f"Exported to {record['file']} with {record['nodes']} nodes and {record['relationships']} relationships in {record['time']} ms")

if __name__ == "__main__":
    # Replace these values with your Neo4j connection details
    uri = "bolt://chiltepin.health.unm.edu:4500"
    user = "neo4j"
    password = "Hello@001"

    exporter = Neo4jExporter(uri, user, password)
    exporter.export_nodes()
    exporter.export_relationships()
    exporter.close()


Before running import code, copy (or move) the export nodes.tsv and relationships.tsv to the import directory of new container 
(Currently /home/www/htdocs/x/cfde/distillery/data/CKG/import)

In [7]:
from neo4j import GraphDatabase
import csv

class Neo4jImporter:
    def __init__(self, uri, user, password, batch_size=1000):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.batch_size = batch_size
        
    def close(self):
        self.driver.close()
    
    def import_nodes(self):
        query = """
        CALL apoc.periodic.iterate(
            "LOAD CSV WITH HEADERS FROM 'file:///nodes.tsv' AS row FIELDTERMINATOR ',' RETURN row", 
            "WITH row, trim(row.Node_label) AS label
            CALL apoc.create.node([label], {
                CUI: trim(row.CUI),
                SAB: trim(row.SAB),
                node_code: trim(row.Code),
                node_label: trim(row.node_label),
                node_synonyms: trim(row.Synonyms)
            }) YIELD node
            RETURN node;",
            {batchSize:500000, parallel:true}
        )
        YIELD batches, total
        RETURN batches, total
        """

        with self.driver.session() as session:
            result = session.run(query)
            for record in result:
                print(f"Imported {record['total']} nodes in {record['batches']} batches.")

    def delete_duplicate_nodes(self):
        query = """
        MATCH (c)
        WHERE c.CUI IS NOT NULL
        WITH c.CUI AS cui, COLLECT(c) AS cnodes
        WHERE SIZE(cnodes) > 1
        FOREACH (n IN TAIL(cnodes) | DETACH DELETE n);
        """

        with self.driver.session() as session:
            result = session.run(query)
            for record in result:
                print("result: ", record)
        
        print("Duplicate Nodes deleted")

    def create_node_index(self):
        queries = [
        "CREATE INDEX IF NOT EXISTS FOR (n:EFO) ON (n.CUI);",
        "CREATE INDEX IF NOT EXISTS FOR (n:NCBI) ON (n.CUI);",
        "CREATE INDEX IF NOT EXISTS FOR (n:Protein) ON (n.CUI);",
        "CREATE INDEX IF NOT EXISTS FOR (n:Gene) ON (n.CUI);",
        "CREATE INDEX IF NOT EXISTS FOR (n:GO) ON (n.CUI);",
        "CREATE INDEX IF NOT EXISTS FOR (n:Disease) ON (n.CUI);",
        "CREATE INDEX IF NOT EXISTS FOR (n:Compound) ON (n.CUI);",
        "CREATE INDEX IF NOT EXISTS FOR (n:Tissue) ON (n.CUI);",
        ]

        with self.driver.session() as session:
            for query in queries:
                result = session.run(query)
                for record in result:
                    print("result: ", record)
        
        print("created indexes")
        
    def fetch_and_process_relationships(self, output_file='relationships_processed.csv'):
        query = """
        LOAD CSV WITH HEADERS FROM 'file:///relationships.tsv' AS row FIELDTERMINATOR ','
        RETURN DISTINCT trim(row.sourceLabel) AS source_label, trim(row.targetLabel) AS target_label, trim(row.relation) AS relation, trim(row.source) AS source, trim(row.target) AS target
        """
        
        relationships = []
        batch_count = 0
        
        with self.driver.session() as session:
            result = session.run(query)
            for record in result:
                relationships.append({
                    'source_label': record['source_label'],
                    'target_label': record['target_label'],
                    'relation': record['relation'],
                    'source': record['source'],
                    'target': record['target']
                })
                if len(relationships) >= self.batch_size:
                        self.save_to_file(relationships, output_file, batch_count)
                        batch_count += 1
                        relationships.clear()

            if relationships:
                self.save_to_file(relationships, output_file, batch_count)
        print("relationships created")
        return output_file

    def save_to_file(self, relationships, output_file, batch_count):
        mode = 'a' if batch_count > 0 else 'w'
        with open(output_file, mode, newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['source_label', 'target_label', 'relation', 'source', 'target', 'SAB'])
            if batch_count == 0:
                writer.writeheader()
            for relationship in relationships:
                writer.writerow(relationship)

        print("Written to file: ", output_file, batch_count)

    def run_queries(self, session, relationships):
        for relationship in relationships:
            source_label = relationship['source_label']
            target_label = relationship['target_label']
            relation = relationship['relation']
            source = relationship['source']
            target = relationship['target']
            sab = relationship['SAB']
            
            query = f"""
            MATCH (source:{source_label} {{CUI: '{source}'}}), (target:{target_label} {{CUI: '{target}'}})
            CREATE (source)-[:{relation} {{SAB: '{relationship["SAB"]}'}}]->(target)
            """
            
            session.run(query)

    def generate_and_run_cql_queries_from_file(self, input_file='relationships_processed.csv'):
        with self.driver.session() as session, open(input_file, mode='r') as file:
            reader = csv.DictReader(file)
            relationships = []
            for relationship in reader:
                relationships.append(relationship)
                if len(relationships) >= self.batch_size:
                    self.run_queries(session, relationships)
                    relationships.clear()

            if relationships:
                self.run_queries(session, relationships)

            print("Batch of queries run: ", len(relationships))

    def cleanup(output_file='relationships_processed.csv'):
        import os
        
        if os.path.exists(output_file):
            os.remove(output_file)


    def fetch_and_process_relationships(self, output_dir='relationships_processed'):
        query = """
        LOAD CSV WITH HEADERS FROM 'file:///relationships.tsv' AS row FIELDTERMINATOR ','
        RETURN DISTINCT trim(row.sourceLabel) AS source_label, trim(row.targetLabel) AS target_label, trim(row.relation) AS relation, trim(row.source) AS source, trim(row.target) AS target, trim(row.SAB) as SAB
        """
        import os
        relationships = {}
        if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

        with self.driver.session() as session:
            result = session.run(query)
            for record in result:
                key = (record['source_label'], record['target_label'], record['relation'])
                if key not in relationships:
                    relationships[key] = []
                relationships[key].append({
                    'source': record['source'],
                    'target': record['target'], 
                    "SAB": record["SAB"]
                })

                if len(relationships) >= 1000:
                    for key, value_list in relationships.items():
                        source_label, target_label, relation = key
                        filename = f"{output_dir}/{source_label}_{target_label}_{relation}.csv"

                        mode = 'a' if os.path.exists(filename) else 'w'
                        with open(filename, mode, newline='') as file:
                            writer = csv.writer(file)
                            if mode == 'w':
                                writer.writerow(['source_label', 'target_label', 'relation', 'source', 'target', 'SAB'])
                            for value in value_list:
                                writer.writerow([source_label, target_label, relation, value['source'], value['target'], value['SAB']])
        
                    print("relationships processed", len(relationships))
                    relationships.clear()
        
        # for key, value_list in relationships.items():
        #     source_label, target_label, relation = key
        #     filename = f"{output_dir}/{source_label}_{target_label}_{relation}.csv"
        #     with open(filename, mode='w', newline='') as file:
        #         writer = csv.writer(file)
        #         writer.writerow(['source_label', 'target_label', 'relation', 'source', 'target', 'SAB'])
        #         for value in value_list:
        #             writer.writerow([source_label, target_label, relation, value['source'], value['target'], value['SAB']])
        print("all relationships written")
        return output_dir

    def generate_and_run_cql_queries_from_files(self, input_dir='processed_relationships'):
        with self.driver.session() as session:
            for filename in os.listdir(input_dir):
                print("Processing", filename)
                if filename.endswith('.csv'):
                    filepath = os.path.join(input_dir, filename)
                    
                    with open(filepath, mode='r') as file:
                        reader = csv.DictReader(file)
                        rows = [row for row in reader]
                    
                    if rows:
                        source_label = rows[0]['source_label']
                        target_label = rows[0]['target_label']
                        relation = rows[0]['relation']
                        
                        query = f"""
                        UNWIND $rows AS row
                        MATCH (source:{source_label} {{CUI: row.source}}), (target:{target_label} {{CUI: row.target}})
                        CREATE (source)-[:{relation} {{SAB: row.SAB}}]->(target)
                        """
                        
                        session.run(query, parameters={'rows': rows})
                    
                    # Delete the file after processing
                    os.remove(filepath)

            # Optionally, delete the directory if empty
            if not os.listdir(input_dir):
                os.rmdir(input_dir)

uri = "bolt://chiltepin.health.unm.edu:7687"
user = "neo4j"
password = "Hello@001"

importer = Neo4jImporter(uri, user, password)

In [ ]:


# importer.import_nodes()
# importer.create_node_index()
# importer.delete_duplicate_nodes()

importer.fetch_and_process_relationships()



relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships processed 1000
relationships 

In [ ]:
importer.generate_and_run_cql_queries_from_files()

In [ ]:
importer.cleanup()

In [ ]:
importer.close()